# Web Automation and Information Retrieval with Python / Automação Web e Busca de Informações com Python

#### Challenge / Desafio: 

We work for an import company, and the prices of our products are linked to the exchange rates of:
Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:

- Dólar
- Euro

We need to automatically fetch the exchange rates for these currencies from the internet and calculate the prices we should charge for our products, considering a profit margin that we have in our database.

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.


DataBase / Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

In [46]:
import requests
import json

#import the prices with API/ importa os valores com API
cotacoes = requests.get('https://economia.awesomeapi.com.br/json/All')
cotacoes_dic = cotacoes.json()
#print(cotacoes_dic)

cotacao_dolar=cotacoes_dic['USD']['bid']
cotacao_euro = cotacoes_dic['EUR']['bid']


print('Dolar compra :{}'.format(cotacao_dolar))
print('Euro compra :{}'.format(cotacao_euro))


Dolar compra :5.2866
Euro compra :5.7476


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [43]:
# Import the data base using pandas/ importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.4,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.0,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.7,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.7,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.9,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.0,4804.800


- Atualizando os preços e o cálculo do Preço Final

In [44]:
# Calculate the new price of each product/ Recalcular o preço de cada produto
# Update the cotation / atualizar a cotação
# For the lines dolar or euro in the column "moedas" /  nas linhas onde na coluna "Moeda" = Dólar e Euro
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

# Update the price in Reais / atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Update the final price / atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]


tabela["Cotação"] = tabela["Cotação"].map("R$ {:,.2f}".format)
tabela["Preço de Compra"] = tabela["Preço de Compra"].map("R$ {:,.2f}".format)
tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R$ {:,.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,R$ 5.29,"R$ 5,287.55",1.4,"R$ 7,402.57"
1,Carro Renault,4500.00,Euro,R$ 5.75,"R$ 25,864.20",2.0,"R$ 51,728.40"
2,Notebook Dell,899.99,Dólar,R$ 5.29,"R$ 4,758.79",1.7,"R$ 8,089.94"
3,IPhone,799.00,Dólar,R$ 5.29,"R$ 4,224.79",1.7,"R$ 7,182.15"
4,Carro Fiat,3000.00,Euro,R$ 5.75,"R$ 17,242.80",1.9,"R$ 32,761.32"
5,Celular Xiaomi,480.48,Dólar,R$ 5.29,"R$ 2,540.59",2.0,"R$ 5,081.17"


### Agora vamos exportar a nova base de preços atualizada

In [45]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False)